# pandas

Now that we know how to get our data in, let's look a little more at `pandas` and what it can do. To do this, let's bring in some data from Zillow.

The data frame we will be working with contains data from Zillow.com, the housing web site, on the number of homes, the median value of homes, etc. at-risk for flooding due to rising sea levels.

You can find many different Zillow data sets [here](https://www.zillow.com/research/data/). The “uw” here stands for “under water” in a very literal sense. I first thought they were talking about mortgages!

In [1]:
import numpy as np
import pandas as pd

uw = pd.read_csv('https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/zestimatesAndCutoffs_byGeo_uw_2017-10-10_forDataPage.csv')

## Looking at our data

Let's peak at this data. You can do that below, or using the `.info` method.

In [2]:
uw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   RegionType                    2610 non-null   object 
 1   RegionName                    2610 non-null   object 
 2   StateName                     2609 non-null   object 
 3   MSAName                       1071 non-null   object 
 4   AllHomes_Tier1                2610 non-null   float64
 5   AllHomes_Tier2                2610 non-null   float64
 6   AllHomes_Tier3                2610 non-null   float64
 7   AllHomes_AllTiers             2610 non-null   float64
 8   UWHomes_Tier1                 2610 non-null   float64
 9   UWHomes_Tier2                 2610 non-null   int64  
 10  UWHomes_Tier3                 2610 non-null   float64
 11  UWHomes_AllTiers              2610 non-null   float64
 12  UWHomes_TotalValue_Tier1      2610 non-null   float64
 13  UWH

There's a concept in data analysis called **tidy data**. The characteristics of tidy data are:

- Each variable forms a column.
- Each observation forms a row.
- Each type of observational unit forms a table.

```{figure} ../images/04-tidy.png
---
name: 04-tidy.png
align: center
---
Tidy data.
```

The first thing to note about this data is that there are several tiers. Some numbers are at the national level, some at the region, some at the state, and some at the MSA. **Tidy data** refers to one row, one observation, across different variables (columns) and, possibly, across time. When you have multiple observations of the same firm, state, person, etc. over time, we call that panel data. Panel data is tidy too.

This data isn’t tidy! Note that some RegionTypes (e.g. Nation, State) are actually summaries of other rows. This is bit confusing and we need to be mindful of this different levels when using the data.

Before we get to that, just a few more methods to look at the data.

First, let's look at the first five rows.

In [3]:
uw.head()

,RegionType,RegionName,StateName,MSAName,AllHomes_Tier1,AllHomes_Tier2,AllHomes_Tier3,AllHomes_AllTiers,UWHomes_Tier1,UWHomes_Tier2,...,UWHomes_TotalValue_Tier3,UWHomes_TotalValue_AllTiers,UWHomes_MedianValue_AllTiers,AllHomes_Tier1_ShareUW,AllHomes_Tier2_ShareUW,AllHomes_Tier3_ShareUW,AllHomes_AllTiers_ShareUW,UWHomes_ShareInTier1,UWHomes_ShareInTier2,UWHomes_ShareInTier3
0,Nation,United States,NaN,NaN,35461549.0,35452941.0,35484532.0,106399022.0,594672.0,542681,...,5.974490e+11,9.159540e+11,310306.0,0.016769,0.015307,0.020458,0.017512,0.319149,0.291246,0.389605
1,State,Alabama,Alabama,NaN,546670.0,520247.0,491300.0,1558217.0,2890.0,2766,...,2.583039e+09,3.578203e+09,270254.0,0.005287,0.005317,0.010271,0.006868,0.270043,0.258456,0.471501
2,State,California,California,NaN,3060171.0,3076238.0,3169584.0,9305993.0,8090.0,14266,...,2.666638e+10,4.449536e+10,860373.0,0.002644,0.004637,0.005611,0.004313,0.201550,0.355415,0.443035
3,State,Connecticut,Connecticut,NaN,333904.0,334688.0,336254.0,1004846.0,4434.0,3807,...,8.689480e+09,1.152170e+10,442036.0,0.013279,0.011375,0.021677,0.015455,0.285512,0.245138,0.469350
4,State,Delaware,Delaware,NaN,102983.0,127573.0,127983.0,358539.0,4105.0,2574,...,1.498013e+09,2.801847e+09,241344.5,0.039861,0.020177,0.025675,0.027793,0.411942,0.258304,0.329754


`NaN` is how `pandas` shows missing values. So, that first row is `RegionType == 'Nation'` and `RegionName == 'UnitedStates'`. `StateName` and `MSAName` are undefined, since this row is **all of the data added up**. We usually don't want to have summary statistics in the same DataFrame as underlying data! What a mess!

We can also look at the last five rows.

In [4]:
uw.tail()

,RegionType,RegionName,StateName,MSAName,AllHomes_Tier1,AllHomes_Tier2,AllHomes_Tier3,AllHomes_AllTiers,UWHomes_Tier1,UWHomes_Tier2,...,UWHomes_TotalValue_Tier3,UWHomes_TotalValue_AllTiers,UWHomes_MedianValue_AllTiers,AllHomes_Tier1_ShareUW,AllHomes_Tier2_ShareUW,AllHomes_Tier3_ShareUW,AllHomes_AllTiers_ShareUW,UWHomes_ShareInTier1,UWHomes_ShareInTier2,UWHomes_ShareInTier3
2605,Zip,98592,Washington,NaN,464.0,470.0,493.0,1427.0,8.0,29,...,73407109.0,82715412.0,436490.5,0.017241,0.061702,0.273834,0.120533,0.046512,0.168605,0.784884
2606,Zip,98595,Washington,NaN,558.0,571.0,598.0,1727.0,119.0,104,...,13761194.0,40365702.0,131441.0,0.213262,0.182137,0.095318,0.162131,0.425000,0.371429,0.203571
2607,Zip,98612,Washington,NaN,365.0,376.0,409.0,1150.0,91.0,79,...,14406737.0,50284001.0,230495.0,0.249315,0.210106,0.092910,0.180870,0.437500,0.379808,0.182692
2608,Zip,32081,Florida,NaN,1318.0,1328.0,1350.0,3996.0,91.0,42,...,91112117.0,132505542.0,385196.0,0.069044,0.031627,0.097778,0.066316,0.343396,0.158491,0.498113
2609,Zip,33578,Florida,NaN,5975.0,4537.0,3232.0,13744.0,8.0,6,...,56852018.0,59115506.0,363355.0,0.001339,0.001322,0.038366,0.010041,0.057971,0.043478,0.898551


Looking at this data, I think it really has four levels that describe the numbers. The columns `RegionType`, `RegionName`, `StateName`, and `MSAName` define unique rows of data. Again, note that these values can be missing. For example, row 2606 is Zip, 98595, Washington, but no `MSAName`. Why? MSA stands for Metropolitan Statistical Area and cover multiple zip codes.

Let’s think about the other variable definitions a bit. Zillow places homes into three tiers based on the home’s estimated market value, with Tier3 being the highest value. AllHomes refers to all homes in that tier or across all tiers. UWHomes refers to homes that are at risk for flooding. Note that there are some variables that are counts, some that are dollar values, and others that are percentages.



## Filtering our data

Let's try **filtering** our data and keeping just the MSA observations. The text calls this **complex selection**. See pg. 132.

Here's some [help on filtering](https://datagy.io/filter-pandas/) data with `pandas`.

In [5]:
uw[uw['RegionType'] == 'MSA'].head(15)

,RegionType,RegionName,StateName,MSAName,AllHomes_Tier1,AllHomes_Tier2,AllHomes_Tier3,AllHomes_AllTiers,UWHomes_Tier1,UWHomes_Tier2,...,UWHomes_TotalValue_Tier3,UWHomes_TotalValue_AllTiers,UWHomes_MedianValue_AllTiers,AllHomes_Tier1_ShareUW,AllHomes_Tier2_ShareUW,AllHomes_Tier3_ShareUW,AllHomes_AllTiers_ShareUW,UWHomes_ShareInTier1,UWHomes_ShareInTier2,UWHomes_ShareInTier3
251,MSA,"Aberdeen, WA",Washington,"Aberdeen, WA",11606.0,11751.0,11749.0,35106.0,4119.0,2450,...,1.734113e+08,7.791588e+08,88327.0,0.354903,0.208493,0.065537,0.209053,0.561248,0.333833,0.104919
252,MSA,"Astoria, OR",Oregon,"Astoria, OR",4745.0,5228.0,5568.0,15541.0,811.0,376,...,9.138231e+07,3.620604e+08,237912.0,0.170917,0.071920,0.040409,0.090856,0.574363,0.266289,0.159348
253,MSA,"Egg Harbor Township, NJ",New Jersey,"Egg Harbor Township, NJ",34185.0,34117.0,34088.0,102390.0,9116.0,9575,...,5.576646e+09,1.030631e+10,264441.0,0.266667,0.280652,0.252200,0.266510,0.334066,0.350887,0.315047
254,MSA,"Baltimore, MD",Maryland,"Baltimore, MD",298318.0,312603.0,316502.0,927423.0,2293.0,2910,...,3.698818e+09,5.153664e+09,336801.0,0.007686,0.009309,0.021861,0.013071,0.189160,0.240059,0.570780
255,MSA,"East Falmouth, MA",Massachusetts,"East Falmouth, MA",45345.0,45358.0,45258.0,135961.0,1651.0,1395,...,4.205784e+09,5.349099e+09,588650.0,0.036410,0.030755,0.073004,0.046705,0.260000,0.219685,0.520315
256,MSA,"Baton Rouge, LA",Louisiana,"Baton Rouge, LA",78815.0,79581.0,78731.0,237127.0,2214.0,1904,...,5.392566e+08,1.219648e+09,180953.5,0.028091,0.023925,0.024895,0.025632,0.364265,0.313261,0.322474
257,MSA,"Bay City, TX",Texas,"Bay City, TX",4161.0,4252.0,4030.0,12443.0,195.0,351,...,1.175738e+08,1.964820e+08,179753.5,0.046864,0.082549,0.107692,0.078759,0.198980,0.358163,0.442857
258,MSA,"Beaumont, TX",Texas,"Beaumont, TX",49842.0,47829.0,44616.0,142287.0,1182.0,1043,...,1.466850e+08,3.249066e+08,83413.0,0.023715,0.021807,0.018312,0.021379,0.388560,0.342867,0.268573
259,MSA,"Bellingham, WA",Washington,"Bellingham, WA",29239.0,30413.0,30956.0,90608.0,2033.0,812,...,7.000724e+08,1.184436e+09,211919.0,0.069530,0.026699,0.040283,0.045162,0.496823,0.198436,0.304741
260,MSA,"Boston, MA",Massachusetts,"Boston, MA",409592.0,409966.0,410140.0,1229698.0,15948.0,15210,...,2.851453e+10,4.266696e+10,570168.0,0.038936,0.037101,0.052509,0.042851,0.302653,0.288648,0.408699


How about just North Carolina MSAs? I'll use the bigger DataFrame and just show the observations, rather than create a new DataFrame.

In [7]:
uw[(uw['RegionType'] == 'MSA') & (uw['StateName'] == 'North Carolina')].head(15)

,RegionType,RegionName,StateName,MSAName,AllHomes_Tier1,AllHomes_Tier2,AllHomes_Tier3,AllHomes_AllTiers,UWHomes_Tier1,UWHomes_Tier2,...,UWHomes_TotalValue_Tier3,UWHomes_TotalValue_AllTiers,UWHomes_MedianValue_AllTiers,AllHomes_Tier1_ShareUW,AllHomes_Tier2_ShareUW,AllHomes_Tier3_ShareUW,AllHomes_AllTiers_ShareUW,UWHomes_ShareInTier1,UWHomes_ShareInTier2,UWHomes_ShareInTier3
274,MSA,"Elizabeth City, NC",North Carolina,"Elizabeth City, NC",5887.0,5882.0,5868.0,17637.0,2198.0,1364,...,4.784827e+08,8.833370e+08,138246.0,0.373365,0.231894,0.283401,0.296252,0.420670,0.261053,0.318278
277,MSA,"Greenville, NC",North Carolina,"Greenville, NC",22042.0,19844.0,18836.0,60722.0,71.0,34,...,6.786239e+06,1.776378e+07,99179.5,0.003221,0.001713,0.001221,0.002108,0.554688,0.265625,0.179688
288,MSA,"Jacksonville, NC",North Carolina,"Jacksonville, NC",17623.0,19413.0,17989.0,55025.0,792.0,1123,...,6.370269e+08,1.029168e+09,264070.5,0.044941,0.057848,0.083329,0.062045,0.231986,0.328940,0.439074
292,MSA,"Kill Devil Hills, NC",North Carolina,"Kill Devil Hills, NC",3194.0,3213.0,3486.0,9893.0,1461.0,1584,...,9.128899e+08,1.650730e+09,280409.0,0.457420,0.492997,0.542456,0.498939,0.295989,0.320908,0.383104
297,MSA,"Morehead City, NC",North Carolina,"Morehead City, NC",13555.0,13843.0,12927.0,40325.0,3195.0,3308,...,2.316937e+09,3.449861e+09,244691.0,0.235706,0.238966,0.359712,0.276578,0.286470,0.296602,0.416928
302,MSA,"New Bern, NC",North Carolina,"New Bern, NC",13637.0,13409.0,12440.0,39486.0,1364.0,1138,...,5.442292e+08,8.553945e+08,163643.0,0.100022,0.084868,0.132235,0.105025,0.328912,0.274415,0.396672
334,MSA,"Washington, NC",North Carolina,"Washington, NC",1289.0,1069.0,1043.0,3401.0,255.0,201,...,5.513269e+07,9.990351e+07,116902.5,0.197828,0.188026,0.182167,0.189944,0.394737,0.311146,0.294118
336,MSA,"Wilmington, NC",North Carolina,"Wilmington, NC",37100.0,40034.0,37475.0,114609.0,3367.0,3339,...,4.265842e+09,6.761845e+09,383067.0,0.090755,0.083404,0.127578,0.100228,0.293114,0.290676,0.416210


Notice how I needed to put `()` around each condition. Python is testing to see if each of these are `True` or `False` and then filtering our data accordingly. Pgs. 132 - 135 of our text show you more about this type of **boolean** filtering.